In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

/upb/users/s/ssmirnov/profiles/unix/imt/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


In [3]:
train_labels

array([4, 9, 6, ..., 2, 4, 4], dtype=int32)

In [4]:
train_labels[:,None]

array([[4],
       [9],
       [6],
       ...,
       [2],
       [4],
       [4]], dtype=int32)

In [5]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)

  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  # np.arrange returns evenly spaced values within a given interval

  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

In [6]:
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [7]:
tf.truncated_normal([128 * 128, 10])

<tf.Tensor 'truncated_normal:0' shape=(16384, 10) dtype=float32>

In [8]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000
learning_rate = 1e-1

graph = tf.Graph()
with graph.as_default():
    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
    
    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
    
    

In [9]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])


In [10]:
with tf.Session(graph=graph) as session:
    
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases.
    tf.global_variables_initializer().run()
    print('Initialized')
    
    for step in range(num_steps):
        
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        
        if (step % 100 == 0):
            
            print('Loss at step %d: %f' % (step, l))
            print('Training accuracy: %.1f%%' % accuracy(
            predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
            print('Validation accuracy: %.1f%%' % accuracy(
                valid_prediction.eval(), valid_labels))
            print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 18.096191
Training accuracy: 5.1%
Validation accuracy: 5.4%
Test accuracy: 5.0%
Loss at step 100: 3.992060
Training accuracy: 54.2%
Validation accuracy: 54.1%
Test accuracy: 60.2%
Loss at step 200: 3.129519
Training accuracy: 63.9%
Validation accuracy: 63.7%
Test accuracy: 70.3%
Loss at step 300: 2.763943
Training accuracy: 67.6%
Validation accuracy: 67.2%
Test accuracy: 74.1%
Loss at step 400: 2.540694
Training accuracy: 69.5%
Validation accuracy: 69.1%
Test accuracy: 75.9%
Loss at step 500: 2.379937
Training accuracy: 71.0%
Validation accuracy: 70.5%
Test accuracy: 77.1%
Loss at step 600: 2.254575
Training accuracy: 72.1%
Validation accuracy: 71.3%
Test accuracy: 78.0%
Loss at step 700: 2.151704
Training accuracy: 72.8%
Validation accuracy: 72.0%
Test accuracy: 78.6%
Loss at step 800: 2.064133
Training accuracy: 73.4%
Validation accuracy: 72.5%
Test accuracy: 79.0%


In [11]:
# Let's now switch to stochastic gradient descent training instead, which is much faster.
# The graph will be similar, except that instead of holding all the training data into a constant node, 
# we create a Placeholder node which will be fed actual data at every call of session.run().
# The advantage of Using SGD is that now we operate with only a subset of training data at the time moment (batch_size), 
# that allows eventually to use the whole dataset and to limited ourself with "train_subset = 10000"

batch_size = 128

graph = tf.Graph()
with graph.as_default():
    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size*image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    
    weights = tf.Variable(
    tf.truncated_normal([image_size*image_size, num_labels]))
    
    biases = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation
    
    logits = tf.matmul(tf_train_dataset,weights) + biases
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
    
    
    # Optimizer
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    # Predictions for the training, validation, and test data.

    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)   

In [12]:
# The offset gives an arithmetic sequence within each epoch and different offsets can be obtained among different epochs
# The expression for the offset generates a cyclic group of numbers.
# These offsets make each mini-batch different from each other not only within each epoch but also among epochs.

for i in range(10):
    offset = (i * batch_size) % (train_labels.shape[0] - batch_size)
    print(offset)

0
128
256
384
512
640
768
896
1024
1152


In [13]:
# Now lets compute it. Do not forget to increase the number of iterations (num_steps) since now we operate at one time
# not with the whole dataset but with the randomly chosen batch of data (batch_size)

num_steps = 13001

with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    print("Initializer")
    
    for step in range(num_steps):
        
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_labels}
        _, l, predictions = session.run([optimizer,loss,train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initializer
Minibatch loss at step 0: 13.473099
Minibatch accuracy: 18.8%
Validation accuracy: 17.3%
Minibatch loss at step 500: 2.927255
Minibatch accuracy: 64.8%
Validation accuracy: 71.9%
Minibatch loss at step 1000: 1.828910
Minibatch accuracy: 74.2%
Validation accuracy: 74.1%
Minibatch loss at step 1500: 1.694937
Minibatch accuracy: 75.8%
Validation accuracy: 75.3%
Minibatch loss at step 2000: 1.640110
Minibatch accuracy: 75.8%
Validation accuracy: 75.6%
Minibatch loss at step 2500: 1.923465
Minibatch accuracy: 75.0%
Validation accuracy: 75.6%
Minibatch loss at step 3000: 1.140934
Minibatch accuracy: 78.1%
Validation accuracy: 76.1%
Minibatch loss at step 3500: 1.062628
Minibatch accuracy: 77.3%
Validation accuracy: 76.4%
Minibatch loss at step 4000: 1.443967
Minibatch accuracy: 74.2%
Validation accuracy: 76.6%
Minibatch loss at step 4500: 1.260072
Minibatch accuracy: 78.9%
Validation accuracy: 76.8%
Minibatch loss at step 5000: 1.293436
Minibatch accuracy: 76.6%
Validation accura

In [14]:
# Now tha task is to the logistic regression example with SGD into a 1-hidden layer neural network 
# with rectified linear units nn.relu() and 1024 hidden nodes. This model should improve your validation / test accuracy.

In [19]:
batch_size = 128
learning_rate = 1e-1
n_hidden = 1024

graph = tf.Graph()
with graph.as_default():
    
    # Input data. Exactly the same as above (two placeholders to handle baches of trainig samples)
    tf_train_dataset = tf.placeholder(tf.float32, shape = (batch_size, image_size*image_size))
    
    tf_train_labels = tf.placeholder(tf.float32, shape = (batch_size,num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables
    
    weights_1 = tf.Variable(tf.truncated_normal([image_size*image_size,n_hidden]))
    bias_1 = tf.Variable(tf.zeros(n_hidden))
    
    weights_2 = tf.Variable(tf.truncated_normal([n_hidden,num_labels]))
    
    bias_2 = tf.Variable(tf.zeros(num_labels))
    
    # Compute activation at the first layer 
    
    pre_activation_1 = tf.matmul(tf_train_dataset, weights_1) + bias_1
    activation_1 = tf.nn.relu(pre_activation_1)
    
    # Now compute logits
    
    logits = tf.matmul(activation_1,weights_2) + bias_2
    
    # continue with defining loss, optimizer etc.. the same as before
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels,logits=logits))
    
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    
    # Make predictions
    
    train_prediction = tf.nn.softmax(logits)
    
    valid_activation_1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_1) + bias_1)
    valid_prediction = tf.nn.softmax(tf.matmul(valid_activation_1,weights_2) + bias_2)
    
    
    test_prediction_1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_1) + bias_1)
    test_prediction = tf.nn.softmax(tf.matmul(test_prediction_1,weights_2) + bias_2)
    

In [20]:
# Run the same iterator as above

num_steps = 3001

with tf.Session(graph=graph) as session:
    
    tf.global_variables_initializer().run()
    print("Initializer")
    
    for step in range(num_steps):
        
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch.
        batch_data = train_dataset[offset:(offset+batch_size),:]
        batch_labels = train_labels[offset:(offset+batch_size),:]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it.
        feed_dict = {tf_train_dataset:batch_data, tf_train_labels:batch_labels}
        _, l, predictions = session.run([optimizer,loss,train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(valid_prediction.eval(), valid_labels))
    print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initializer
Minibatch loss at step 0: 264.974060
Minibatch accuracy: 14.1%
Validation accuracy: 27.1%
Minibatch loss at step 500: 25.149525
Minibatch accuracy: 76.6%
Validation accuracy: 79.3%
Minibatch loss at step 1000: 14.043876
Minibatch accuracy: 77.3%
Validation accuracy: 81.1%
Minibatch loss at step 1500: 13.234638
Minibatch accuracy: 76.6%
Validation accuracy: 79.9%
Minibatch loss at step 2000: 8.233657
Minibatch accuracy: 83.6%
Validation accuracy: 82.6%
Minibatch loss at step 2500: 8.149773
Minibatch accuracy: 82.0%
Validation accuracy: 83.5%
Minibatch loss at step 3000: 4.688102
Minibatch accuracy: 89.8%
Validation accuracy: 84.1%
Test accuracy: 90.5%
